In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

dataset = pd.read_csv("dataset.csv")

print(dataset.isnull().sum())

ImportError: cannot import name 'LogisticRegression' from 'sklearn.preprocessing' (C:\Users\user\anaconda3\Lib\site-packages\sklearn\preprocessing\__init__.py)

In [12]:
dataset

,Temperature,Heart Rate,Oxygen Saturation,Respiratory Rate,CO2 Level,Activity Level,Skin Temperature,Cough Count,Room Humidity,Air Quality Index,Noise Level,Proximity,COVID-19 Status
0,37.646441,89,99.755407,16,741,1812,34.774361,10,47,98,67,137,Suspect
1,38.145568,75,96.146920,12,513,9085,36.716540,2,33,121,24,42,Suspect
2,37.808290,75,90.770160,14,467,3883,36.133021,5,57,60,52,152,Suspect
3,37.634650,75,95.416080,16,395,9826,34.351239,2,56,2,56,121,Suspect
4,37.270964,81,91.380077,16,832,9875,37.684310,9,40,83,58,193,Suspect
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,38.177763,83,92.573203,20,698,46,37.348060,13,57,12,27,112,Suspect
99996,37.522562,79,99.371052,13,701,4075,34.284446,19,31,24,24,15,Suspect
99997,38.430399,72,93.413615,12,412,2882,36.362479,10,49,151,68,31,Suspect
99998,37.653097,62,92.375070,22,991,2605,37.754327,4,46,91,60,104,Suspect


In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['COVID-19 Status'] = le.fit_transform(dataset['COVID-19 Status'])

In [14]:
X = dataset.drop(columns="COVID-19 Status", axis=1)
Y = dataset["COVID-19 Status"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=41)

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
smote = SMOTE(random_state=42)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train_scaled, Y_train)

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression 

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  
    'solver': ['liblinear', 'lbfgs'], 
    'max_iter': [200, 500, 1000]
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_resampled, Y_train_resampled)

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'max_iter': [200, 500, 1000],
                         'solver': ['liblinear', 'lbfgs']},
             scoring='accuracy')

In [21]:
best_model = grid_search.best_estimator_
print(grid_search.best_params_)

{'C': 0.1, 'max_iter': 200, 'solver': 'lbfgs'}


In [22]:
y_train_pred = best_model.predict(X_train_scaled)
y_test_pred = best_model.predict(X_test_scaled)

In [23]:
train_accuracy = accuracy_score(Y_train, y_train_pred)
test_accuracy = accuracy_score(Y_test, y_test_pred)

In [24]:
train_accuracy

0.9101625

In [25]:
test_accuracy

0.91

In [28]:
fake_input = np.array([
    [
        np.random.uniform(36, 40),  
        np.random.randint(60, 100),  
        np.random.uniform(90, 100),  
        np.random.randint(12, 24),  
        np.random.randint(400, 1000), 
        np.random.randint(0, 10),  
        np.random.uniform(34, 38),  
        np.random.randint(0, 20),  
        np.random.randint(30, 60),  
        np.random.randint(0, 150),  
        np.random.randint(40, 80),  
        np.random.randint(50, 150),  
    ]
])

fake_input_df = pd.DataFrame(fake_input, columns=[
    "Temperature", "Heart Rate", "Oxygen Saturation", "Respiratory Rate", 
    "CO2 Level", "Activity Level", "Skin Temperature", "Cough Count", 
    "Room Humidity", "Air Quality Index", "Noise Level", "Proximity"
])

In [29]:
prediction = best_model.predict(fake_input_df)

if prediction[0] == 1:
    print("suspect.")
else:
    print("not suspect.")

not suspect.


C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
